In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/emotions-dataset-for-nlp/val.txt
/kaggle/input/emotions-dataset-for-nlp/test.txt
/kaggle/input/emotions-dataset-for-nlp/train.txt


In [2]:
texts = []
labels = []

with open(r"/kaggle/input/emotions-dataset-for-nlp/train.txt", encoding='UTF8') as f:
    contents = f.read()
    contents = contents.split("\n")
    for content in contents:
        try:
            texts.append(content.split(";")[0])
            labels.append(content.split(";")[1])
        except:
            
            if len(texts) > len(labels):
                texts.pop(len(texts)-1)
        
data = {'text': texts,
       'label': labels}
  
print(len(texts), len(labels))
    
# Create DataFrame
train = pd.DataFrame(data)

16000 16000


In [3]:
texts = []
labels = []

with open(r"/kaggle/input/emotions-dataset-for-nlp/val.txt", encoding='UTF8') as f:
    contents = f.read()
    contents = contents.split("\n")
    for content in contents:
        try:
            texts.append(content.split(";")[0])
            labels.append(content.split(";")[1])
        except:
            
            if len(texts) > len(labels):
                texts.pop(len(texts)-1)
        
data = {'text': texts,
       'label': labels}
  
print(len(texts), len(labels))
    
# Create DataFrame
val = pd.DataFrame(data)

2000 2000


In [4]:
texts = []
labels = []

with open(r"/kaggle/input/emotions-dataset-for-nlp/test.txt", encoding='UTF8') as f:
    contents = f.read()
    contents = contents.split("\n")
    for content in contents:
        try:
            texts.append(content.split(";")[0])
            labels.append(content.split(";")[1])
        except:
            
            if len(texts) > len(labels):
                texts.pop(len(texts)-1)
        
data = {'text': texts,
       'label': labels}
  
print(len(texts), len(labels))
    
# Create DataFrame
test = pd.DataFrame(data)

2000 2000


In [5]:
set(train['label'])

{'anger', 'fear', 'joy', 'love', 'sadness', 'surprise'}

In [6]:
train[train["label"] == 'love']["text"].values[0:4]

array(['i am ever feeling nostalgic about the fireplace i will know that it is still on the property',
       'i feel romantic too',
       'i can t let go of that sad feeling that i want to be accepted here in this first home of mine',
       'i ate i could feel a gentle tingle throughout almost as if i was feeling the healing taking place at a cellular level'],
      dtype=object)

In [7]:
HT = {}

for i, entry in enumerate(set(train['label'])):
    HT[entry] = i

HT

{'love': 0, 'surprise': 1, 'joy': 2, 'fear': 3, 'anger': 4, 'sadness': 5}

In [8]:
temp = []

for entry in train['label']:
    temp.append(HT[entry])

train.drop(labels=['label'], inplace=True, axis=1)
train['label'] = temp

train.head()

,text,label
0,i didnt feel humiliated,5
1,i can go from feeling so hopeless to so damned...,5
2,im grabbing a minute to post i feel greedy wrong,4
3,i am ever feeling nostalgic about the fireplac...,0
4,i am feeling grouchy,4


In [9]:
temp = []

for entry in val['label']:
    temp.append(HT[entry])

val.drop(labels=['label'], inplace=True, axis=1)
val['label'] = temp

val.head()

,text,label
0,im feeling quite sad and sorry for myself but ...,5
1,i feel like i am still looking at a blank canv...,5
2,i feel like a faithful servant,0
3,i am just feeling cranky and blue,4
4,i can have for a treat or if i am feeling festive,2


In [10]:
temp = []

for entry in test['label']:
    temp.append(HT[entry])

test.drop(labels=['label'], inplace=True, axis=1)
test['label'] = temp

test.head()

,text,label
0,im feeling rather rotten so im not very ambiti...,5
1,im updating my blog because i feel shitty,5
2,i never make her separate from me because i do...,5
3,i left with my bouquet of red and yellow tulip...,2
4,i was feeling a little vain when i did this one,5


In [11]:
# Extract Feature Vectors

from sklearn import feature_extraction

vectorizer = feature_extraction.text.CountVectorizer()

train_vectors = vectorizer.fit_transform(train["text"])

val_vectors = vectorizer.transform(val["text"])
test_vectors = vectorizer.transform(test["text"])

In [12]:
from sklearn import tree

clf = tree.DecisionTreeClassifier()
clf = clf.fit(train_vectors, train['label'])

clf.score(train_vectors, train['label']), clf.score(val_vectors, val['label']), clf.score(test_vectors, test['label'])

(0.998125, 0.8655, 0.853)

In [13]:
# # Hyper-Parameter Tuning using Optuna
# import optuna

# def objective(trial):
    
#     # Set the Parameters
# #     max_depth = int(trial.suggest_int('max_depth', 1, 3000))
#     criterion = trial.suggest_categorical('criterion', ['gini', 'entropy', 'log_loss'])
#     min_samples_split = int(trial.suggest_int('min_samples_split', 2, 101))
#     min_samples_leaf = int(trial.suggest_int('min_samples_leaf', 1, 101))
#     max_feature = trial.suggest_categorical('max_feature', ['auto', 'sqrt', 'log2'])
#     splitter = trial.suggest_categorical('splitter', ['best', 'random'])
    
#     try:
#         model = tree.DecisionTreeClassifier(criterion=criterion, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, max_features=max_feature, splitter=splitter)
#         model.fit(train_vectors, train['label'])

#         return model.score(val_vectors, val['label'])
    
#     except:
        
#         return 0

In [14]:
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=400)

In [15]:
# study.best_params, study.best_value

In [16]:
clf = tree.DecisionTreeClassifier(criterion='gini', min_samples_split=91, min_samples_leaf= 1, max_features='sqrt', splitter='random')
clf = clf.fit(train_vectors, train['label'])

clf.score(train_vectors, train['label']), clf.score(val_vectors, val['label']), clf.score(test_vectors, test['label'])

(0.8225625, 0.7675, 0.737)

In [17]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()

model.fit(train_vectors, train['label'])

model.score(train_vectors, train['label']), model.score(val_vectors, val['label']), model.score(test_vectors, test['label'])

(0.998125, 0.8755, 0.8675)

In [18]:
# def objective(trial):
    
#     # Set the Parameters
#     n_estimator = trial.suggest_int('n_estimators', 2, 4000)
# #     max_depth = int(trial.suggest_int('max_depth', 1, 320))
#     criterion = trial.suggest_categorical('criterion', ['gini', 'entropy', 'log_loss'])
#     min_samples_split = int(trial.suggest_int('min_samples_split', 2, 100))
#     min_samples_leaf = int(trial.suggest_int('min_samples_leaf', 1, 100))
#     max_feature = trial.suggest_categorical('max_feature', ['auto', 'sqrt', 'log2'])
    
#     try:
#         model = RandomForestClassifier(n_estimators=n_estimator, criterion=criterion, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, max_features=max_feature)
#         model.fit(train_vectors, train['label'])

#         return model.score(val_vectors, val['label'])
    
#     except:
        
#         return 0

In [19]:
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=400)

In [20]:
# study.best_params

In [21]:
train_vectors.shape, val_vectors.shape, test_vectors.shape

((16000, 15186), (2000, 15186), (2000, 15186))

In [22]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential, layers
from tensorflow.keras.utils import to_categorical

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [23]:
model = Sequential()

model.add(layers.Dense(10000, activation='relu', input_shape=(train_vectors.shape[1],)))

model.add(layers.Dense(10000, activation='relu'))

model.add(layers.Dense(1000, activation='relu'))

model.add(layers.Dense(1000, activation='relu'))

model.add(layers.Dense(100, activation='relu'))

model.add(layers.Dense(100, activation='relu'))

model.add(layers.Dense(6, activation='softmax'))

model.summary()

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss=tf.keras.losses.CategoricalCrossentropy())

model.fit(train_vectors, to_categorical(train['label'], num_classes=6), batch_size = 320, epochs=20)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10000)             151870000 
                                                                 
 dense_1 (Dense)             (None, 10000)             100010000 
                                                                 
 dense_2 (Dense)             (None, 1000)              10001000  
                                                                 
 dense_3 (Dense)             (None, 1000)              1001000   
                                                                 
 dense_4 (Dense)             (None, 100)               100100    
                                                                 
 dense_5 (Dense)             (None, 100)               10100     
                                                                 
 dense_6 (Dense)             (None, 6)                 6

In [24]:
preds = model.predict(val_vectors)

num_correct = 0

for i, pred in enumerate(preds):
    temp = pred.tolist()
#     print(val['label'].iloc[i], temp.index(max(temp)))

    if val['label'].iloc[i] == temp.index(max(temp)):
        num_correct += 1

num_correct = (num_correct/len(preds))

print(num_correct)

63/63 [==============================] - 7s 111ms/step
0.8475


In [25]:
preds = model.predict(test_vectors)

num_correct = 0

for i, pred in enumerate(preds):
    temp = pred.tolist()
#     print(val['label'].iloc[i], temp.index(max(temp)))

    if test['label'].iloc[i] == temp.index(max(temp)):
        num_correct += 1

num_correct = (num_correct/len(preds))

print(num_correct)

63/63 [==============================] - 7s 111ms/step
0.8375
